In [1]:
import keras
from keras import backend
from keras.models import load_model

2024-03-04 18:33:07.631600: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 18:33:07.725516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Example of taking feature extractor from siamese network

In [2]:
# Model creation
def _create_model(nneurons, nfilters, ndropout, npool):
    # 4 layers
    inputs = keras.Input((400, 400, 3))
    
    x = keras.layers.Conv2D(nneurons[0], (nfilters[0], nfilters[0]), padding="same", activation="relu")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[0], npool[0]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[1], (nfilters[1], nfilters[1]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[1], npool[1]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[2], (nfilters[2], nfilters[2]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[2], npool[2]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[3], (nfilters[3], nfilters[3]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[3], npool[3]), data_format='channels_last')(x)
    x = keras.layers.Dropout(ndropout[0])(x)

    pooledOutput = keras.layers.GlobalAveragePooling2D()(x)
    pooledOutput = keras.layers.Dense(nneurons[4])(pooledOutput)
    outputs = keras.layers.Dense(nneurons[5])(pooledOutput)

    model = keras.Model(inputs, outputs)
    return model

def _euclidean_distance(vectors):
    (featA, featB) = vectors
    squared = backend.square(featA-featB)
    return squared

def siamese_model(nneurons, nfilters, ndropout, npool):
    feature_extractor_model = _create_model(nneurons, nfilters, ndropout, npool)
    imgA = keras.Input(shape=(400, 400, 3))
    imgB = keras.Input(shape=(400, 400, 3))
    featA = feature_extractor_model(imgA)
    featB = feature_extractor_model(imgB)
    distance = keras.layers.Lambda(_euclidean_distance)([featA, featB])
    outputs = keras.layers.Dense(1, activation="sigmoid")(distance)
    model = keras.Model(inputs=[imgA, imgB], outputs=outputs)
    return model

In [3]:
# Create siamese model
nneurons = [32, 64, 64, 128, 256, 128]
nfilters = [3, 3, 5, 5]
ndropout = [0.4]
npool = [4, 4, 4, 4]
siamesemodel = siamese_model(nneurons, nfilters, ndropout, npool)

In [4]:
# load weights - note that this includes 2 additional weights for the output layer 
siamesemodel.load_weights('./optimized_weights.h5')

In [5]:
# Save siamese model
# siamesemodel.save('siamese_model.h5')

/home/oku/miniforge3/envs/xai4geo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Create a model to extract feature vectors from weights of the trained model
feature_extractor_model = _create_model(nneurons, nfilters, ndropout, npool)

In [ ]:
# Set the weights of the extractor model to the weights trained in the siamese model.  
feature_extractor_model.set_weights(siamesemodel.weights[:28])

## Compare the extracted model and the loaded model

In [ ]:
load_model('./siamese_model.h5')

In [ ]:
# Load the saved model with the weights
feature_extractor_model_saved = load_model('./feature_extractor_model.h5')

In [ ]:
# Verify that the weights are the same by comparing the sum of the differences
# All differences should be 0
sum_of_diffs = []
for i in range(len(feature_extractor_model_saved.layers)):
    sum_of_diffs.append((feature_extractor_model_saved.weights[i].numpy() - feature_extractor_model.weights[i].numpy()).sum())
sum_of_diffs